## Monitor Data Drift & Violations using SageMaker Model Monitoring
This notebook shows how to:
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata 
* Analyze a training dataset to generate baseline constraints
* Monitor a live endpoint for violations against constraints

---
## Background

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to analyze the data and monitor its quality. In this notebook, you learn how Amazon SageMaker enables these capabilities.


![model-monitor](./model-monitor.jpg)


### Imports

In [1]:
from sagemaker import get_execution_role, session
from sagemaker.model import Model
import sagemaker
import boto3
import json
import re
import os

In [2]:
sagemaker.__version__

'2.16.4.dev0'

### Essentials

In [3]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sklearn-classifier/model-monitor'
# COPY the training job name of the previous training job you ran from the SageMaker Console under Training Jobs
# Let's re-use the Model we trained in Notebook 2 (Built-in Algorithm XGBoost)
training_job_name = 'classifier-2020-11-09-02-18-10-731' # CHANGE THIS
# COPY S3 model artifact location from the console
model_url = 's3://sagemaker-demo-892313895307/clf/model-artifacts/classifier-2020-11-09-02-18-10-731/output/model.tar.gz'
container_image_uri = '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'
role = get_execution_role()

### S3 Paths

In [4]:
data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

code_prefix = '{}/code'.format(prefix)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

Capture path: s3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/datacapture
Report path: s3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/reports


## Capture Real-Time Inference Data from SageMaker Endpoints

In [5]:
model = Model(image_uri=container_image_uri, 
              model_data=model_url, 
              role=role)

### Re-deploy using DataCaptureConfig

To enable data capture for monitoring the model data quality, you specify the new capture option called DataCaptureConfig. You can capture the request payload, the response payload or both with this configuration. The capture config applies to all variants. Go ahead with the deployment.

In [6]:
from sagemaker.model_monitor import DataCaptureConfig
from time import gmtime, strftime

In [7]:
endpoint_name = 'classifier-xgboost-model-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        capture_options=["REQUEST", "RESPONSE"],
                        destination_s3_uri=s3_capture_upload_path)

EndpointName=classifier-xgboost-model-monitor-2020-12-09-23-24-21


In [8]:
data_capture_config.__dict__

{'enable_capture': True,
 'sampling_percentage': 100,
 'destination_s3_uri': 's3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/datacapture',
 'kms_key_id': None,
 'capture_options': ['REQUEST', 'RESPONSE'],
 'csv_content_types': ['text/csv'],
 'json_content_types': ['application/json']}

In [9]:
model.deploy(initial_instance_count=1, 
             instance_type='ml.m5.xlarge', 
             endpoint_name=endpoint_name, 
             data_capture_config=data_capture_config)

---------------!

### Invoke the Deployed Model Monitor Endpoint 


You can now send data to this endpoint to get inferences in real time. Because you enabled the data capture in the previous steps, the request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location you have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 2 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [10]:
from sagemaker.serializers import CSVSerializer
from sagemaker.predictor import Predictor

import pandas as pd
pd.set_option('display.max_colwidth', -1)

import numpy as np
import json
import time

In [11]:
csv_serializer = CSVSerializer()
predictor = Predictor(endpoint_name=endpoint_name, serializer = csv_serializer)

In [12]:
test_df = pd.read_csv('.././DATA/test/test.csv', names=['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [13]:
test_df.head()

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,-0.940089,-0.403964,-0.279542,-0.817379
1,0,-0.502614,-0.665582,0.131742,-0.362450
2,0,1.078473,0.347981,0.228029,-0.817379
3,1,-0.636164,-0.251491,0.587034,-0.817379
4,1,-0.528479,2.037253,1.383463,0.185934


In [14]:
i = 0
for _, row in test_df.iterrows():
    X =[row.bmi, row.diastolic_bp_change, row.systolic_bp_change, row.respiratory_rate]
    payload = np.array(X)
    response = predictor.predict(data=payload)
    print(response.decode('utf-8'))
    if i == 10:
        break
    i += 1

0.1326758712530136
0.16127055883407593
0.006274824496358633
0.0025395755656063557
0.014364791102707386
0.041070837527513504
0.002037754049524665
0.1326758712530136
0.5306588411331177
0.470020592212677
0.041070837527513504


### View captured data

NOTE: Could take a minute here

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [18]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
current_endpoint_capture_prefix

'sklearn-classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-12-09-23-24-21'

In [19]:
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
result.get('Contents')

[{'Key': 'sklearn-classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-12-09-23-24-21/AllTraffic/2020/12/09/23/32-10-196-e802822b-f932-4fa7-95e0-875565f8e24c.jsonl',
  'LastModified': datetime.datetime(2020, 12, 9, 23, 33, 16, tzinfo=tzlocal()),
  'ETag': '"d909ce7a6b837e6395b76a164d8b4db9"',
  'Size': 4878,
  'StorageClass': 'STANDARD'}]

In [20]:
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
sklearn-classifier/model-monitor/datacapture/classifier-xgboost-model-monitor-2020-12-09-23-24-21/AllTraffic/2020/12/09/23/32-10-196-e802822b-f932-4fa7-95e0-875565f8e24c.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [21]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

In [22]:
capture_file = get_obj_body(capture_files[-1])
capture_file[:1000]

'{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-0.9400893790050578,-0.4039637071832271,-0.27954219645020045,-0.8173792379004012","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.1326758712530136","encoding":"CSV"}},"eventMetadata":{"eventId":"c6bfa91a-afb9-4198-a17c-831e061d350d","inferenceTime":"2020-12-09T23:32:10Z"},"eventVersion":"0"}\n{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-0.5026137735439099,-0.6655817166298655,0.13174226914679035,-0.3624501673165596","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.16127055883407593","encoding":"CSV"}},"eventMetadata":{"eventId":"3e334b5a-ce85-472d-9185-e9ba1341cabf","inferenceTime":"2020-12-09T23:32:10Z"},"eventVersion":"0"}\n{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"1.0784731280550373,0

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [23]:
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "-0.9400893790050578,-0.4039637071832271,-0.27954219645020045,-0.8173792379004012",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.1326758712530136",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "c6bfa91a-afb9-4198-a17c-831e061d350d",
    "inferenceTime": "2020-12-09T23:32:10Z"
  },
  "eventVersion": "0"
}


As you can see, each inference request is captured in one line in the jsonl file. The line contains both the input and output merged together. In the example, you provided the ContentType as `text/csv` which is reflected in the `observedContentType` value. Also, you expose the encoding that you used to encode the input and output payloads in the capture format with the `encoding` value.

To recap, you observed how you can enable capturing the input or output payloads to an endpoint with a new parameter. You have also observed what the captured format looks like in Amazon S3. Next, continue to explore how Amazon SageMaker helps with monitoring the data collected in Amazon S3.

## Baseling & Continuous Monitoring

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:
1. Create a baseline with which you compare the realtime traffic. 
1. Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.

In [24]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/baselining/results


### Create a Baselining Job with Training Dataset

In [25]:
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import DefaultModelMonitor

#### Upload Train Set to S3 as Baseline Data

In [26]:
train_data = open('.././DATA/train/train_with_header.csv', 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'train_with_header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(train_data)

In [27]:
default_model_monitor = DefaultModelMonitor(
                            role=role,
                            instance_count=1,
                            instance_type='ml.r5.xlarge',
                            volume_size_in_gb=20,
                            max_runtime_in_seconds=3600,
                        )

In [28]:
default_model_monitor.suggest_baseline(
                            baseline_dataset=baseline_data_uri + '/train_with_header.csv',
                            dataset_format=DatasetFormat.csv(header=True),
                            output_s3_uri=baseline_results_uri,
                            wait=True
                        )


Job Name:  baseline-suggestion-job-2020-12-09-23-34-37-885
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/baselining/data/train_with_header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-892313895307/sklearn-classifier/model-monitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................2020-12-09 23:38:36,303 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:processing-job/baseline-suggestion-job-2020-12-09-23-34-37-885', 'ProcessingJobName': 'baseline-suggestion-job-2020-12-09-23-34-37-885', 'Environment': {'dataset_format': '{"csv": {"header": 

2020-12-09 23:38:45,934 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start nodemanager, return code 1
2020-12-09 23:38:45,935 - bootstrap - INFO - Running command: /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver
2020-12-09 23:38:48,024 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver, return code 1
2020-12-09 23:38:48,025 - DefaultDataAnalyzer - INFO - Total number of hosts in the cluster: 1
2020-12-09 23:38:58,036 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.KryoSerializer /opt/amazon/sagemaker-data-analyzer-1.0-jar-with-dependencies.jar --analytics_input /tmp/spark_job_config.json
2020-12-09 23:38:59 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java 

2020-12-09 23:39:13 INFO  YarnSchedulerBackend$YarnDriverEndpoint:54 - Registered executor NettyRpcEndpointRef(spark-client://Executor) (10.2.242.176:50372) with ID 1
2020-12-09 23:39:13 INFO  BlockManagerMasterEndpoint:54 - Registering block manager algo-1:43197 with 11.9 GB RAM, BlockManagerId(1, algo-1, 43197, None)
2020-12-09 23:39:30 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2020-12-09 23:39:30 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2020-12-09 23:39:30 INFO  DatasetReader:91 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/train_with_header.csv)
2020-12-09 23:39:30 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark

### Explore the generated constraints and statistics

In [29]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get('Key') for report_file in result.get('Contents')]

In [30]:
print('Found Files:')
print("\n ".join(report_files))

Found Files:
sklearn-classifier/model-monitor/baselining/results/constraints.json
 sklearn-classifier/model-monitor/baselining/results/statistics.json


In [31]:
baseline_job = default_model_monitor.latest_baselining_job

In [32]:
baseline_job.__dict__

{'inputs': [<sagemaker.processing.ProcessingInput at 0x7f31e84c2eb8>],
 'outputs': [<sagemaker.processing.ProcessingOutput at 0x7f31e9ae53c8>],
 'output_kms_key': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7f31e6e18a58>,
 'job_name': 'baseline-suggestion-job-2020-12-09-23-34-37-885'}

In [33]:
baseline_job.suggested_constraints().body_dict['monitoring_config']

{'evaluate_constraints': 'Enabled',
 'emit_metrics': 'Enabled',
 'datatype_check_threshold': 1.0,
 'domain_content_threshold': 1.0,
 'distribution_constraints': {'perform_comparison': 'Enabled',
  'comparison_threshold': 0.1,
  'comparison_method': 'Robust'}}

In [34]:
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict['features'])
schema_df.head(10)

name inferred_type  numerical_statistics.common.num_present  \
0  class                Integral      7461                                      
1  bmi                  Fractional    7461                                      
2  diastolic_bp_change  Fractional    7461                                      
3  systolic_bp_change   Fractional    7461                                      
4  respiratory_rate     Fractional    7461                                      

   numerical_statistics.common.num_missing  numerical_statistics.mean  \
0  0                                        1.872403e-01                
1  0                                       -7.367262e-17                
2  0                                        7.053287e-17                
3  0                                        3.392571e-16                
4  0                                        1.166099e-16                

   numerical_statistics.sum  numerical_statistics.std_dev  \
0  1.397000e+03              0.390104                       
1 -5.496714e-13              1.000000                       
2  5.262457e-13              1.000000                       
3  2.531197e-12              1.000000                       
4  8.700263e-13              1.000000                       

   numerical_statistics.min  numerical_statistics.max  \
0  0.000000                  1.000000                   
1 -1.615623                  3.500391                   
2 -1.003436                  5.753651                   
3 -0.927406                  12.648954                  
4 -0.817379                  22.607916                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  numerical_statistics.distribution.kll.buckets  \
0  [{'lower_bound': 0.0, 'upper_bound': 0.1, 'count': 6064.0}, {'lower_bound': 0.1, 'upper_bound': 0.2, 'count': 0.0}, {'lower_bound': 0.2, 'upper_bound': 0.3, 'count': 0.0}, {'lower_bound': 0.3, 'upper_bound': 0.4, 'count': 0.0}, {'lower_bound': 0.4, 'upper_bound': 0.5, 'count': 0.0}, {'lower_bound': 0.5, 'upper_bound': 0.6, 'count': 0.0}, {'lower_bound': 0.6, 'upper_bound': 0.7, 'count': 0.0}, {'lower_bound': 0.7, 'upper_bound': 0.8, 'count': 0.0}, {'lower_bound': 0.8, 'upper_bound': 0.9, 'count': 0.0}, {'lower_bound': 0.9, 'upper_bound': 1.0, 'count': 1397.0}]                                                                                                                                                                                                                                                                                                                                         
1  [{'lower_bound': -1.615623300687038, 'upper_bound': -1.1040218968988618, 'count': 1117.0}, {'lower_bound': -1.1040218968988618, 'upper_bound': -0.5924204931106856, 'count': 1465.0}, {'lower_bound': -0.5924204931106856, 'upper_bound': -0.08081908932250959, 'count': 1230.0}, {'lower_bound': -0.08081908932250959, 'upper_bound': 0.43078231446566684, 'count': 1131.0}, {'lower_bound': 0.43078231446566684, 'upper_bound': 0.9423837182538428, 'count': 1070.0}, {'lower_bound': 0.9423837182538428, 'upper_bound': 1.4539851220420188, 'count': 792.0}, {'lower_bound': 1.4539851220420188, '

In [35]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict['features'])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,class,Integral,1.0,True
1,bmi,Fractional,1.0,False
2,diastolic_bp_change,Fractional,1.0,False
3,systolic_bp_change,Fractional,1.0,False
4,respiratory_rate,Fractional,1.0,False


## Monitoring Schedules

<p><b>Analyzing collected data for data quality issues</b></p>

When you have collected the data above, analyze and monitor the data with Monitoring Schedules

### Create a Schedule

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

In [ ]:
mon_schedule_name = 'clf-xgb-model-monitor-schedule-' + strftime('%Y-%m-%d-%H-%M-%S', gmtime())

default_model_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=default_model_monitor.baseline_statistics(),
    constraints=default_model_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

### Test Model Monitoring using Artificial Traffic
The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [ ]:
from threading import Thread
from time import sleep
import time

In [ ]:
endpoint_name = predictor.endpoint
#endpoint_name = 'classifier-xgboost-model-monitor-2020-11-08-03-43-50'
runtime_client = boto3.client('runtime.sagemaker')

In [ ]:
def invoke_endpoint(endpoint_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType='text/csv', 
                                           Body=payload)
            #prediction = response['Body'].read()

In [ ]:
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, '.././DATA/test/model_monitor_test.csv', runtime_client)

In [ ]:
thread = Thread(target=invoke_endpoint_forever)
thread.start()
# NOTE: You need to stop the kernel to stop the invocations

### Describe and Inspect the Schedule
Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [ ]:
desc_schedule_result = default_model_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

### List executions
The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [ ]:
mon_executions = default_model_monitor.list_executions()

print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer).")
print("We will have to wait till we hit the hour ...") 


In [ ]:
# Wait till you see an execution object in this list before you proceed to the next step
# takes between (60 to 80 mins)
mon_executions

### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - Job exceeded max runtime or was manually stopped.

In [ ]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on ...
latest_execution.describe()

In [ ]:
print("Latest Execution Status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest Execution Result: {}".format(latest_execution.describe()['ExitMessage']))

In [ ]:
report_uri = latest_execution.output.destination
print('Report URI: {}'.format(report_uri))

### List the Generated Reports

In [ ]:
from urllib.parse import urlparse

In [ ]:
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report Bucket: {}'.format(report_bucket))
print('Report Key: {}'.format(report_key))

In [ ]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

### Violations Report

If there are any violations compared to the baseline, they will be listed here.

In [ ]:
violations = default_model_monitor.latest_monitoring_constraint_violations()
violations.__dict__

In [ ]:
constraints_df = pd.json_normalize(violations.body_dict["violations"])

In [ ]:
constraints_df

### Shift in Statistics 

In [ ]:
statistics_shift = default_model_monitor.latest_monitoring_statistics()

In [ ]:
statistics_shift_df = pd.json_normalize(statistics_shift.body_dict["features"])
statistics_shift_df

### Other Commands
We can also start and stop the monitoring schedules.

In [ ]:
# my_default_monitor.stop_monitoring_schedule()
# my_default_monitor.start_monitoring_schedule()

## Delete the Resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
# my_default_monitor.delete_monitoring_schedule()
# time.sleep(60) # actually wait for the deletion

In [ ]:
# predictor.delete_endpoint()

In [ ]:
# predictor.delete_model()